In [17]:
import sys
import os

# Manually set PONI_ROOT and PYTHONPATH
PONI_ROOT = "/home/marmot/Boyang/MEM-Nav"  # Update this if needed
os.environ["PONI_ROOT"] = PONI_ROOT
os.environ["PYTHONPATH"] = os.environ.get("PYTHONPATH", "") + ":" + PONI_ROOT
# Append the path to sys.path (if not already included)
if PONI_ROOT not in sys.path:
    sys.path.append(PONI_ROOT)

import json
import glob
SCENES_ROOT = "/home/marmot/Boyang/MEM-Nav/data/scene_datasets/gibson_semantic"
ACTIVE_DATASET="gibson"
from poni.constants import SPLIT_SCENES

In [18]:
def check_object_room_overlap(obj, room):
    # Object bounding box
    obj_x_min, obj_y_min, obj_z_min = obj['location']
    obj_x_max = obj_x_min + obj['size'][0]
    obj_y_max = obj_y_min + obj['size'][1]
    obj_z_max = obj_z_min + obj['size'][2]

    # Room bounding box
    room_x_min, room_y_min, room_z_min = room['location']
    room_x_max = room_x_min + room['size'][0]
    room_y_max = room_y_min + room['size'][1]
    room_z_max = room_z_min + room['size'][2]

    # Check overlap in each dimension
    overlap_x = max(0, min(obj_x_max, room_x_max) - max(obj_x_min, room_x_min))
    overlap_y = max(0, min(obj_y_max, room_y_max) - max(obj_y_min, room_y_min))
    overlap_z = max(0, min(obj_z_max, room_z_max) - max(obj_z_min, room_z_min))

    # If overlap exists in all dimensions, the object is inside or overlapping the room
    is_overlapping = overlap_x > 0 and overlap_y > 0 and overlap_z > 0

    # Return results
    return {
        "overlap_x": overlap_x,
        "overlap_y": overlap_y,
        "overlap_z": overlap_z,
        "is_overlapping": is_overlapping
    }

In [22]:
scene_paths = sorted(
    glob.glob(
        os.path.join(SCENES_ROOT, "**/*.glb"),
        recursive=True,
    )
)
# Select only scenes that have correspondingsemantics
scene_paths = list(
    filter(
        lambda x: os.path.isfile(x.replace(".glb", "_semantic.ply")), scene_paths
    )
)

# Select only scenes from the train and val splits
valid_scenes = (
    SPLIT_SCENES[ACTIVE_DATASET]["train"] + SPLIT_SCENES[ACTIVE_DATASET]["val"]
)
scene_paths = list(
    filter(lambda x: os.path.basename(x).split(".")[0] in valid_scenes, scene_paths)
)
print(len(scene_paths))
for scene_path in scene_paths:
    env_name = scene_path.split("/")[-1].split('.')[0]
    print(env_name)
    scn_path = scene_path.replace(".glb", ".scn")
    with open(scn_path) as fp:
        scn_data = json.load(fp)

    room_dict = {room['id']: room for room in scn_data['rooms']}

    total_cnt = 0
    overlap_cnt = 0
    no_room_cnt = 0
    for obj in scn_data['objects']:
        total_cnt += 1
        if obj['parent_room']:
            res = check_object_room_overlap(obj, room_dict[obj['parent_room']])
            if res['is_overlapping']:
                overlap_cnt += 1
        else:
            no_room_cnt += 1
        
    print(f"Total objects with corresponding room location overlapping: {overlap_cnt}/{total_cnt}={overlap_cnt/total_cnt}")
    print(f"Total objects: {total_cnt}, objects without room:{no_room_cnt}, objects with correct room:{overlap_cnt}")
    print("--------------------------")

29
Allensville
Total objects with corresponding room location overlapping: 7/36=0.19444444444444445
Total objects: 36, objects without room:0, objects with correct room:7
--------------------------
Beechwood
Total objects with corresponding room location overlapping: 26/108=0.24074074074074073
Total objects: 108, objects without room:0, objects with correct room:26
--------------------------
Benevolence
Total objects with corresponding room location overlapping: 19/51=0.37254901960784315
Total objects: 51, objects without room:0, objects with correct room:19
--------------------------
Coffeen
Total objects with corresponding room location overlapping: 32/113=0.2831858407079646
Total objects: 113, objects without room:0, objects with correct room:32
--------------------------
Collierville
Total objects with corresponding room location overlapping: 19/59=0.3220338983050847
Total objects: 59, objects without room:0, objects with correct room:19
--------------------------
Cosmos
Total obje